<a href="https://colab.research.google.com/github/Singular-Brain/DeepBioLCNet/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ! pip install git+https://github.com/BindsNET/bindsnet.git

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
import torch
import os
import numpy as np

from biolcnet import BioLCNet
from reward import DynamicDopamineInjection
from dataset import ClassSelector, load_datasets
import matplotlib.pyplot as plt

gpu = True
seed = 2045 # The singularity is near!

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


if gpu and torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
else:
    torch.manual_seed(seed)
    device = "cpu"
    if gpu:
        gpu = False

torch.set_num_threads(os.cpu_count() - 1)
print("Running on", device)


n_neurons = 1000
n_classes = 10
neuron_per_class = int(n_neurons/n_classes)


train_hparams = {
    'n_train' : 10,
    'n_test' : 3,
    'n_val' : 3,
    'val_interval' : 5,
    'running_window_length': 250,
}

network_hparams = {
    # net structure
    'epochs': 1,
    'crop_size': 20,
    'neuron_per_c': neuron_per_class,
    'in_channels':1,
    'n_channels1': 16,
    'n_channels2': 0,
    'filter_size1': 5,
    'filter_size2': 0,
    'stride1': 2,
    'stride2': 0,
    'deep': False,
    'n_neurons' : n_neurons,
    'n_classes': n_classes,
    
    # time & Phase
    'dt' : 1,
    'pre_observation': True,
    'has_decision_period': True,
    'observation_period': 256,
    'decision_period': 256,
    'time': 256*3,
    'online_rewarding': False,

    # Nodes
    'theta_plus': 0.05,
    'tc_theta_decay': 1e6,
    'tc_trace':20,
    'trace_additive' : False,
    
    # Learning
    'nu_LC': (0.0001,0.01),
    'nu_Output':0.1,

    # weights
    'wmin': 0.0,
    'wmax': 1.0,
    
    # Inhibition
    'inh_type': 'between_layers',
    'inh_factor_FC': 100,
    'inh_LC': True,
    'inh_factor_LC': 100,
    
    # Normalization
    'norm_factor_LC': 0.25*5*5,
    
    # clamping
    'clamp_intensity': None,

    # Save
    'save_path': 'BioLCNet_Small_filter_16_trained.pth',  # Specify for saving the model (Especially for pre-training the lc layer)
    'load_path': None,
    'LC_weights_path': 'BioLCNet_Small_filter_16.pth', # Specify for loading the pre-trained lc weights

    # Plot:
    'confusion_matrix' : False,
    'lc_weights_vis': False,
    'out_weights_vis': False,
    'lc_convergence_vis': False,
    'out_convergence_vis': False,
}

reward_hparams= {
    'n_labels': n_classes,
    'neuron_per_class': neuron_per_class,
    'variant': 'scalar',
    'tc_reward':0,
    'dopamine_base': 0.0,
    'reward_base': 1.,
    'punishment_base': 1.,
    'sub_variant': 'RPE',
    'td_nu': 0.0005,  #RPE
    'ema_window': 10, #RPE
    }

# Dataset Hyperparameters
mask = None
mask_test = None
n_classes = 10

data_hparams = { 
    'intensity': 128,
    'time': 256*3,
    'crop_size': 20,
    'round_input': False,
}

dataloader, val_loader, test_loader = load_datasets(data_hparams, target_classes=None, mask=mask, mask_test=mask_test)


hparams = {**reward_hparams, **network_hparams, **train_hparams, **data_hparams}
net = BioLCNet(**hparams, reward_fn = DynamicDopamineInjection, gpu = gpu)
net.testing = False
net.fit(dataloader = dataloader, val_loader = val_loader, reward_hparams = reward_hparams, **train_hparams)



### Test
data_hparams = { 
    'intensity': 128,
    'time': 256*2,
    'crop_size': 20,
    'round_input': False,
}
network_hparams['time'] = 256*2
net.time = 256*2
_, _, test_loader = load_datasets(data_hparams, target_classes=None, mask=mask, mask_test=mask_test)
net.testing = True
test_acc = net.evaluate(test_loader, train_hparams['n_test'],train_hparams['n_test'],250)
print("Test accuracy: " + "{:.2f}".format(test_acc) + "%")

Running on cuda
Weights loaded ...
Previous model not found! Training from the beginning...

tensor([  0,   0,   0,   0,   0,   0,   0,   0, 256,   0])
tensor([   0,    0,    0, 1428,    0,    0,    0,    0,    0,    0])
tensor([   0,    0,    0,    0, 2195,    0,    0,    0,    0,    0])
tensor([   0,    0,    0,    0, 1553,    0,    0,    0,    0,    0])
tensor([   0,    0,    0,    0,    0,    0,    0,    0, 3228,    0])
tensor([   0,    0,    0,    0,    0, 3159,    0,    0,    0,    0])
Epoch: 1, Running accuracy: 16.67%
tensor([   0,    0,    0,    0, 1771,    0,    0,    0,    0,    0])
tensor([  0,   0,   0,   0,   0, 336,   0,   0,   0,   0])


KeyboardInterrupt: ignored